# 1. 라이브러리

In [1]:
import numpy as np
import pandas as pd

import os
import math
from glob import glob

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# 2. 지역별 데이터 불러오기

In [2]:
data_day_path = os.listdir('./data/data_day/train')
data_hour_path = os.listdir('./data/data_hour/train')

In [3]:
data_day_path[0] ,data_hour_path[0]

('강릉_day_fianl.csv', '강릉_tm_fianl.csv')

전처리 전 데이터를 폴더로부터 불러옵니다.

수집한 raw data가 지역별 csv 파일로 구분되어 있기 때문에 os.listdir를 활용하여 리스트로 만듭니다.

용도별 전처리 함수를 만들고 마지막에 for문으로 리스트에 포함된 모든 지역별 csv 파일에 적용합니다.

# 3. 일별 데이터의 컬럼별 결측치 처리

In [4]:
def get_columns(word):
    summary = pd.read_excel('./data/전처리_요약.xlsx')
    avg_day = summary[summary['방법'] == word]['일_컬럼'].values
    avg_tm = summary[summary['방법'] == word]['시간_컬럼'].values
    avg_dict = dict(zip(avg_day, avg_tm))
    return avg_dict

앞서 수집한 데이터에는 시간별 raw data와 일별 raw data가 있습니다.

최종적으로는 일별 raw data를 활용하여 데이터 분석을 진행할 것이기 때문에 시간 데이터는 일별 데이터를 보완하기 위해 사용합니다.

예를 들어, 일별 데이터의 평균 기온 컬럼의 결측치를 처리하려면 시간 데이터로부터 해당 일의 24시간 기온의 평균으로 대체하는 것입니다.

위 함수는 이런 식으로 일별 데이터의 모든 컬럼에 대해 특성을 고려하여 전처리하기 위해서 시간 데이터로부터 필요한 데이터를 가져옵니다.

# 4. 시간 데이터 전처리

In [5]:
def hour_fill(data_day_path, data_hour_path):
    df_hour = pd.read_csv('./data/data_hour/train/' + data_hour_path)
    df_day = pd.read_csv('./data/data_day/train/' + data_day_path)

    # 강수 및 적설 데이터 전처리
    temp = ['강수량', '적설', '3시간_신절설']

    for i in temp:
        indexs = df_hour[df_hour[i] == 0.0].index
        for j in range(len(indexs)):
            df_hour[i][indexs[j]] = 0.01  
        df_hour[i] = df_hour[i].fillna(0.0)

    # df_hour 결측 row 채우기
    time_list = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                 '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                 '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

    ab_day_list = []
    ab_time_list = []
    before_time_list = []

    for day in df_day['시간'].unique():
        if len(df_hour[df_hour['날짜'] == day]) == 24:
            pass
        else:
            for time in time_list:
                if time not in df_hour[df_hour['날짜'] == day]['시간'].unique():
                    ab_day_list.append(day)
                    ab_time_list.append(time)
                    # print(f'없는 날짜 : {day}')
                    # print(f'없는 시간 : {time}')

    for before_time in ab_time_list:
        before_time_list.append(time_list[time_list.index(before_time) - 1])

    idx = [df_hour[df_hour['날짜']==ab_day_list[i]][df_hour[df_hour['날짜']==ab_day_list[i]]['시간']==before_time_list[i]].index for i in range(len(ab_day_list))]

    for i in range(len(idx)):
        df_hour.loc[(idx[i].item() + idx[i].item()+1)/2] = np.NaN
        df_hour['날짜'][(idx[i].item() + idx[i].item()+1)/2] = ab_day_list[i]
        df_hour['시간'][(idx[i].item() + idx[i].item()+1)/2] = ab_time_list[i]
        df_hour = df_hour.sort_index()
        df_hour.reset_index(drop=True, inplace=True)

        if all(df_hour.iloc[0, :].isnull()) == False:
            df_hour.iloc[0, :] = df_hour.iloc[0, :].fillna(df_hour.iloc[1, :])
        else:
            df_hour = df_hour.fillna(method='ffill')
        df_hour = df_hour.fillna(method='ffill')

        idx = [df_hour[df_hour['날짜']==ab_day_list[i]][df_hour[df_hour['날짜']==ab_day_list[i]]['시간']==before_time_list[i]].index for i in range(len(ab_day_list))]

    return df_day, df_hour

일별 데이터의 결측치를 처리하기 위해서는 시간 데이터의 결측치 처리가 선행되어야 합니다.

#### ① 강수 및 적설 데이터 전처리

해당 컬럼의 0.0 데이터는 소수점 둘째 자리에서 반올림한 것이므로 0.01로 대체하고 결측치는 0으로 대체합니다.

#### ② 시간 데이터 결측 행 처리

시간 데이터에는 Null값이 아니라 특정 시간대 자체가 없는 경우가 있습니다

해당 시간대의 행을 추가하고 시간 데이터의 연속성을 감안하여 fillna(method='ffill')을 활용하여 결측치를 1시간 전의 데이터로 대체합니다.

만약 결측 시간대가 2011년 1월 1일 00:00라면 1시간 뒤의 데이터인 01:00의 데이터로 대체합니다.

# 5. 일별 데이터 전처리

In [6]:
def data_preprocessing(df_day, df_hour):
    if len(df_day.columns) == 62:
        df_day = df_day.drop(columns = [
            '최저_기온_시각', '최고_기온_시각', '10분_최다강수량_시각', '1시간_최다_강수량_시각', '최대_순간풍속_시각',
            '최대_풍속_시각', '평균_상대습도_시각', '최고_해면기압_시각','최저_해면기압_시각', '1시간_최다_일사_시각',
            '일_최심신적설_시각', '일_최심적설_시각','강수_계속시간','10분_최다강수량', '1시간_최다강수량', '최대_순간풍속',
            '최대_순간_풍속_풍향', '가조시간', '일_최심적설', '최저_초상온도', '0.5m_지중온도', '1.0m_지중온도', '1.5m_지중온도',
            '3.0m_지중온도', '5.0m_지중온도', '합계_대형증발량', '9-9강수','합계_소형증발량'
        ])      

    # 컬럼 추가
    df_day['체감온도_여름'] = 0.0
    df_day['체감온도_겨울'] = 0.0

    df_hour['체감온도_여름'] = 0.0
    df_hour['체감온도_겨울'] = 0.0
    df_hour['풍속2'] = 0.0
    df_hour['풍속2'] = df_hour['풍속'] * 3600 / 1000

    # df_hour 계절별 체감온도
    for i in range(len(df_hour)):
        # df_hour 겨울_체감온도
        Ta = df_hour['기온'][i]
        V = df_hour['풍속2'][i]
        df_hour['체감온도_겨울'][i] = 13.12 + 0.6215 * Ta - 11.37 * (V**(0.16)) + 0.3965 * (V**(0.16)) * Ta

        # df_hour 여름_체감온도
        RH = df_hour['습도'][i]
        Tw = Ta * math.atan(0.151977 * ((RH + 8.313659)**(1/2))) + math.atan(Ta + RH) - math.atan(RH - 1.67633) + (0.00391838 * (RH**(3/2)) * math.atan(0.023101 * RH)) - 4.686035
        df_hour['체감온도_여름'][i] = -0.2442 + 0.55399 * Tw + 0.45535 * Ta - 0.0022 * (Tw**2) + 0.00278 * Tw * Ta + 3.5

    # 계절별 체감온도
    for i in range(len(df_day)):
        df_day['체감온도_여름'][i] = round(df_hour[['체감온도_여름']].iloc[24*i:24*(i+1), :].max(), 2)
        df_day['체감온도_겨울'][i] = round(df_hour[['체감온도_겨울']].iloc[24*i:24*(i+1), :].min(), 2)

    # 결측치 처리
    dicts = get_columns('최저')
    for key, values in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for i in range(len(indexs)):
            time = df_day[df_day[key].isnull()]['시간'][indexs[i]]
            df_day[key][indexs[i]] = df_hour[df_hour['날짜'] == time][values].min()

    dicts = get_columns('최고')
    for key, values in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for i in range(len(indexs)):
            time = df_day[df_day[key].isnull()]['시간'][indexs[i]]
            df_day[key][indexs[i]] = df_hour[df_hour['날짜'] == time][values].max()

    dicts = get_columns('합')
    for key, values in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for i in range(len(indexs)):
            time = df_day[df_day[key].isnull()]['시간'][indexs[i]]
            df_day[key][indexs[i]] = df_hour[df_hour['날짜'] == time][values].sum()

    dicts = get_columns('평균')
    for key, values in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for i in range(len(indexs)):
            time = df_day[df_day[key].isnull()]['시간'][indexs[i]]
            df_day[key][indexs[i]] = df_hour[df_hour['날짜'] == time][values].mean()

    dicts = get_columns('합*36')
    for key, values in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for i in range(len(indexs)):
            time = df_day[df_day[key].isnull()]['시간'][indexs[i]]
            df_day[key][indexs[i]] = df_hour[df_hour['날짜'] == time][values].sum() * 36

    dicts = get_columns('강수')
    for i in dicts.keys():
        indexs = df_day[df_day[i] == 0.0].index
        for j in range(len(indexs)):
            df_day[i][indexs[j]] = 0.01  
        df_day[i] = df_day[i].fillna(0.0)

    df_day['1시간_최다_일사량'] = df_day['1시간_최다_일사량'].fillna(0.0)

    dicts = get_columns('조건')
    for key, value in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for j in range(len(indexs)):
            day = df_day['시간'][indexs[j]]
            df_temp = df_hour[df_hour['날짜'] == day]
            max_value = df_temp[key[-5:-3]].max()
            if len(df_temp[df_temp[key[-5:-3]] == max_value][value].values) == 0:
                pass
            else:
                result = df_temp[df_temp[key[-5:-3]] == max_value][value].values[0]
                df_day[key][indexs[j]] = result

    dicts = get_columns('최빈')
    for key, value in dicts.items():
        indexs = df_day[df_day[key].isnull()].index
        for j in range(len(indexs)):
            day = df_day['시간'][indexs[j]]
            df_temp = df_hour[df_hour['날짜'] == day]
            result = df_temp[value].value_counts().keys()[0]
            df_day[key][indexs[j]] = result

    dicts = get_columns('구름')
    for key, value in dicts.items():
        idxes = df_day[df_day[key].isnull()].index
        for i in idxes:
            try:
                df_day[key][i] = round(df_hour.loc[[24*i+3, 24*i+6, 24*i+9, 24*i+12, 24*i+15, 24*i+18, 24*i+21, 24*i+24], [value]].mean(), 1)
            except:
                print(f'{key} 컬럼 {i}번째 인덱스에서 에러 발생')

    return df_day

#### ① 자연재난별 데이터 분석을 위해 필요한 지표를 선정합니다.

불필요한 지표로 판단한 기준은 다음과 같습니다.

- 분석 대상이 되는 자연재난과 무관한 지표

- 결측치가 너무 많아서 결측치 처리 기준을 정하기 어려운 지표

- 최고/최저 지표의 시각을 보여주는 지표

- 다른 지표와 의미가 중복되는 지표

#### ② 체감온도 지표를 추가합니다.

자연재난 중 폭염과 한파는 그 판단 기준이 체감온도입니다.

여름 체감온도(5월 ~ 9월)과 그 외 기간의 체감온도는 계산 공식이 다르기 때문에 시기에 맞게 적용합니다.

#### ③ 앞서 정의한 get_columns 함수를 활용하여 컬럼별 결측치를 처리합니다.

예를 들어, 최저 기온의 결측치를 대체하려면 get_columns('최저'), 최고 기온의 결측치를 대체하려면 get_columns('최고')를 입력합니다.

# 6. 자연재난 라벨링

In [7]:
def labeling(df_day, df_hour, start, end):
    # 컬럼 추가
    df_day['계절'] = '겨울철'
    df_day['체감온도'] = df_day['체감온도_여름']
    df_day['열대야'] = 0.0
    df_day['여름_폭염'] = 0.0
    df_day['겨울_한파'] = 0.0
    df_day['호우'] = 0.0
    df_day['대설'] = 0.0

    df_hour['열대야'] = 0.0
    df_hour['겨울_한파'] = 0.0
    df_hour['24시간_신적설'] = 0.0
    df_hour['호우'] = 0.0
    df_hour['대설'] = 0.0

    # 계절 컬럼 추가
    for year in range(start, end+1):
        for i in range(len(df_day['시간'])):
            if df_day['시간'][i] >= f'{year}-05-01' and df_day['시간'][i] <= f'{year}-09-30':
                df_day['계절'][i] = '여름철'

    # 계절별 체감온도 선택
    for i in range(len(df_day['시간'])):
        if df_day['계절'][i] == '겨울철':
            df_day['체감온도'][i] = df_day['체감온도_겨울'][i]

    # 열대야
    for i in range(len(df_hour['시간'])):
        if i % 24 == 17:
            if all(df_hour['기온'][i:i+16] >= 25.0):
                df_hour['열대야'][i:i+16] = 1

    for i in range(len(df_day)):
        if df_hour['열대야'][24*i + 18] == 1:
            df_day['열대야'][i] = 1

    # 폭염
    for i in range(len(df_day['시간'])):
        if df_day['계절'][i] == '여름철':
            if df_day['체감온도'][i] >= 33.0:
                df_day['여름_폭염'][i] = 1        

    #한파
    for i in range(len(df_hour['시간'])):
        if i % 24 == 2:
            if any(df_hour['기온'][i:i+7] <= -12.0):
                df_hour['겨울_한파'][i:i+7] = 1

    for i in range(len(df_day)):
        if df_hour['겨울_한파'][24*i + 3] == 1:
            df_day['겨울_한파'][i] = 1

    # 강수 및 적설
    for i in range(len(df_hour)-23):
        df_hour['24시간_신적설'][i+23] = df_hour['3시간_신절설'].iloc[i:i+24].sum()

    for i in range(len(df_hour)):
        if df_hour['강수량'][i] >= 20:
            df_hour['호우'][i] = 1
        if df_hour['24시간_신적설'][i] >= 5:
            df_hour['대설'][i] = 1

    for i in range(len(df_day)):
        day = df_day['시간'][i]
        if df_hour[df_hour['날짜'] == day]['호우'].sum() >= 1.0:
            df_day['호우'][i] = 1

    for i in range(len(df_day)):
        day = df_day['시간'][i]
        if df_hour[df_hour['날짜'] == day]['대설'].sum() >= 1.0:
            df_day['대설'][i] = 1

    return df_day

시간 데이터와 일별 데이터의 전처리가 완료되었다면 마지막으로 자연재난 라벨링을 합니다.

#### ① 자연재난 컬럼 추가

분석 대상이 되는 폭염, 한파, 호우, 대설 컬럼을 추가하고 기본값을 0으로 입력합니다.

#### ② 열대야 및 24시간 신적설 컬럼 추가

폭염 데이터 분석에 필요한 열대야 컬럼을 추가하고 시간 데이터를 활용하여 라벨링을 합니다.

대설 라벨링에 필요한 24시간 신적설 컬럼을 추가하고 시간 데이터를 활용하여 값을 계산합니다.

#### ③ 라벨링

- 폭염 : 일별 데이터에서 추가한 체감온도 지표가 폭염의 기준을 충족한다면 1을 입력

- 한파 : 일별 데이터에서 추가한 체감온도 지표가 한파의 기준을 충족한다면 1을 입력

- 호우 : 시간당 강수량이 20 이상일 경우 1을 입력

- 대설 : 24시간 신적설 지표가 5 이상일 경우 1을 입력

기준을 충족하지 않으면 모두 0을 입력하여 0과 1로 라벨링을 완료합니다.

# 7. 전처리 함수 실행

In [8]:
for i in range(len(data_day_path)):
    try:
        print(f'▷ {data_day_path[i][:-14]} 시작 {datetime.now().time()}')
        df_day, df_hour = hour_fill(data_day_path[i], data_hour_path[i])
        df_hour.to_csv('./data/preprocessing/data_hour/train/' + df_hour['지점명'].unique()[0] + '_hour_preprocessing.csv', index=False)
        print(f'hour_fill 완료 {datetime.now().time()}')

        df_day = data_preprocessing(df_day, df_hour)
        df_day.to_csv('./data/preprocessing/data_day/train/' + df_day['지점명'].unique()[0] + '_day_preprocessing.csv', index=False)
        print(f'data_preprocessing 완료 {datetime.now().time()}')

        df_day = labeling(df_day, df_hour, 2011, 2020)
        df_day.to_csv('./data/final_labeling/train/' + df_day['지점명'].unique()[0] + '_day_final.csv', index=False)
        print(data_day_path[i][:-14] + ' 완료 ' + f'{datetime.now().time()}\n')

        datetime.now().time()
    except:
        print(data_day_path[i][:-14] + ' 오류 ' + f'{datetime.now().time()}\n')

▷ 강릉 시작 10:31:04.148471
hour_fill 완료 10:31:20.559563
data_preprocessing 완료 10:32:10.981635
강릉 완료 10:33:08.947367

▷ 강진군 시작 10:33:08.947367
강진군 오류 10:34:34.381499

▷ 강화 시작 10:34:34.381499
hour_fill 완료 10:34:59.156731
data_preprocessing 완료 10:37:46.176348
강화 완료 10:38:43.914691

▷ 거제 시작 10:38:43.914691
hour_fill 완료 10:39:02.953332
data_preprocessing 완료 10:41:45.219360
거제 완료 10:42:43.278376

▷ 거창 시작 10:42:43.278376
hour_fill 완료 10:42:59.573566
data_preprocessing 완료 10:45:38.350450
거창 완료 10:46:36.281810

▷ 경주시 시작 10:46:36.281810
hour_fill 완료 10:48:06.291305
data_preprocessing 완료 10:48:57.955725
경주시 완료 10:49:56.402910

▷ 고산 시작 10:49:56.402910
hour_fill 완료 10:50:14.070125
data_preprocessing 완료 10:52:17.214686
고산 완료 10:53:15.239146

▷ 고창 시작 10:53:15.239146
hour_fill 완료 10:53:46.712355
data_preprocessing 완료 10:55:48.695976
고창 완료 10:56:46.699108

▷ 고창군 시작 10:56:46.699108
hour_fill 완료 10:57:12.156123
data_preprocessing 완료 10:58:02.646240
고창군 완료 10:59:00.873385

▷ 고흥 시작 10:59:00.873385
hour_fill 완

data_preprocessing 완료 14:30:25.303431
창원 완료 14:31:21.792591

▷ 천안 시작 14:31:21.792591
hour_fill 완료 14:31:38.053092
data_preprocessing 완료 14:34:16.330809
천안 완료 14:35:13.174063

▷ 철원 시작 14:35:13.174063
hour_fill 완료 14:35:30.872436
data_preprocessing 완료 14:37:32.549471
철원 완료 14:38:29.474332

▷ 청송군 시작 14:38:29.474332
hour_fill 완료 14:38:56.013450
data_preprocessing 완료 14:39:46.516961
청송군 완료 14:40:44.117039

▷ 청주 시작 14:40:44.117039
hour_fill 완료 14:41:00.490375
data_preprocessing 완료 14:42:29.943262
청주 완료 14:43:27.164859

▷ 추풍령 시작 14:43:27.164859
hour_fill 완료 14:43:43.397705
data_preprocessing 완료 14:45:50.594848
추풍령 완료 14:46:48.451930

▷ 춘천 시작 14:46:48.451930
hour_fill 완료 14:47:04.398862
평균_전운량 컬럼 3652번째 인덱스에서 에러 발생
평균_중하층운량 컬럼 3652번째 인덱스에서 에러 발생
data_preprocessing 완료 14:47:51.509831
춘천 완료 14:48:50.145141

▷ 충주 시작 14:48:50.145141
hour_fill 완료 14:49:07.119202
data_preprocessing 완료 14:51:46.381728
충주 완료 14:52:43.323284

▷ 태백 시작 14:52:43.323284
hour_fill 완료 14:52:59.676648
data_preprocessing 완료 14

앞서 정의한 함수를 for문을 활용하여 모든 지역별 csv 파일에 적용합니다.

# 8. 지역별 데이터 통합

In [9]:
file_path = glob('./data/final_labeling/train/*.csv')

df = pd.DataFrame()

for i in file_path:
    df_temp = pd.read_csv(i)
    df = pd.concat([df, df_temp])

df.drop(columns = ['일기현상', '계절', '체감온도_겨울', '체감온도_여름'], inplace=True)
df['시간'] = pd.to_datetime(df['시간'])
df.reset_index(drop=True, inplace=True)

df['시간'] = df['시간'].astype('str')
temp = df.시간.str.split('-', expand=True)

# 연도-월-일 컬럼 생성
df.insert(3, '연도', temp.iloc[:, 0])
df.insert(4, '월', temp.iloc[:, 1])
df.insert(5, '일', temp.iloc[:, 2])

df['연도'] = df['연도'].astype(int)
df['월'] = df['월'].astype(int)
df['일'] = df['일'].astype(int)

# 계절 컬럼 추가
df.insert(6, '계절', np.NaN)

for i in range(len(df)):
    if 3<= df['월'][i] <= 5:
        df['계절'][i] = '봄'
    elif 6<= df['월'][i] <= 8:
        df['계절'][i] = '여름'
    elif 9<= df['월'][i] <= 11:
        df['계절'][i] = '가을'
    else:
        df['계절'][i] = '겨울'

In [10]:
df.to_csv('./data/visualization_train.csv', index=False)

추후 시각화 과정의 편의성을 위해서 지역별 데이터를 하나로 통합한 데이터를 생성합니다.